In [3]:
import pandas as pd
import numpy as np
import joblib

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline


csv_path = 'superhost.csv'    # 여기에 absolute path
# CSV 읽기
df = pd.read_csv(
    csv_path,
    header=0,        # 첫 줄을 컬럼명으로 사용
    index_col='id',  # 인덱스 컬럼으로 id 지정
    encoding='utf-8-sig'
)

# 1. 타겟 및 피처 정의
TARGET = 'host_is_superhost'

strategy_cols = ['amenities_cnt', 'availability_365', 'price', 'host_about_length_group', 'room_type','name_length_group', 'description_length_group',
                 'host_has_profile_pic', 'host_response_time_score','type_amenity_score','common_amenity_score',
                 'host_acceptance_rate_score', 'host_identity_verified','is_long_term', 'accommodates']

X = df[strategy_cols]
y = df[TARGET].astype(int)

# 2. 열 타입 분리
categorical_cols = X.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# 3. 전처리 파이프라인 구성
preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(handle_unknown='ignore', drop=None), categorical_cols)
], remainder='passthrough')  # 수치형은 그대로 통과

# 4. 전체 파이프라인 구성
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier(
        n_estimators=1000,
        max_depth=30,
        min_samples_split=15,
        min_samples_leaf=10,
        random_state=42,
        class_weight='balanced'
    ))
])

# 5. 학습 데이터 분할 및 모델 학습
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
pipeline.fit(X_train, y_train)

proba = pipeline.predict_proba(X)[:, 1]
print("최대 예측 확률:", proba.max())
print("상위 10개 확률:", sorted(proba, reverse=True)[:10])

proba = pipeline.predict_proba(X_train)[:, 1]
idx = proba.argmax()
print("최대 예측 확률:", proba[idx])
print("상위 10개 확률:", sorted(proba, reverse=True)[:10])
print("\n최대 확률 row의 feature 값:")
print(X_train.iloc[idx])

최대 예측 확률: 0.9551734777411631
상위 10개 확률: [np.float64(0.9551734777411631), np.float64(0.9495232975734985), np.float64(0.948161903186157), np.float64(0.9479109969845085), np.float64(0.9477034068176335), np.float64(0.9448529931520346), np.float64(0.9445799202411178), np.float64(0.9430131530163884), np.float64(0.9428614048341629), np.float64(0.9425255400061279)]
최대 예측 확률: 0.9551734777411631
상위 10개 확률: [np.float64(0.9551734777411631), np.float64(0.9495232975734985), np.float64(0.948161903186157), np.float64(0.9479109969845085), np.float64(0.9448529931520346), np.float64(0.9445799202411178), np.float64(0.9430131530163884), np.float64(0.9428614048341629), np.float64(0.9425255400061279), np.float64(0.9422511236838137)]

최대 확률 row의 feature 값:
amenities_cnt                              22
availability_365                          206
price                                    53.0
host_about_length_group                 empty
room_type                     Entire home/apt
name_length_group          

In [ ]:
import pandas as pd
import numpy as np
import joblib

from sklearn.model_selection import train_test_split, GridSearchCV # GridSearchCV 추가
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score # 모델 평가 지표 확인용

# --- 0. 데이터 로드 ---
# 'superhost.csv' 파일의 절대 경로를 지정해주세요.
# 예: csv_path = 'C:/Users/HY/Documents/GitHub/advanced_project/hayoung/data/superhost.csv'
csv_path = 'superhost.csv'  # 현재 스크립트와 같은 디렉토리에 있다면 상대 경로도 가능

try:
    df = pd.read_csv(
        csv_path,
        header=0,
        index_col='id',
        encoding='utf-8-sig'
    )
    print("CSV 파일을 성공적으로 로드했습니다.")
except FileNotFoundError:
    print(f"오류: '{csv_path}' 파일을 찾을 수 없습니다. 경로를 확인해주세요.")
    exit() # 파일이 없으면 스크립트 종료

# 1. 타겟 및 피처 정의
TARGET = 'host_is_superhost'

strategy_cols = [
    'amenities_cnt', 'availability_365', 'price',
    'host_about_length_group', 'room_type', 'name_length_group', 'description_length_group',
    'host_has_profile_pic', 'host_response_time_score', 'type_amenity_score', 'common_amenity_score',
    'host_acceptance_rate_score', 'host_identity_verified', 'is_long_term', 'accommodates'
]

# 결측치 확인 및 제거 (필요하다면)
# df.dropna(subset=strategy_cols + [TARGET], inplace=True)
# print(f"결측치 처리 후 데이터 크기: {df.shape}")

X = df[strategy_cols]
y = df[TARGET].astype(int) # 타겟 변수는 정수형으로

# 2. 열 타입 분리
# OneHotEncoder는 bool 타입을 자동으로 처리하지 못할 수 있으므로, object/category로 명시적으로 변환
for col in ['host_has_profile_pic', 'host_identity_verified', 'is_long_term']:
    if col in X.columns:
        X[col] = X[col].astype(object) # object 타입으로 변환하여 categorical_cols에 포함되도록 함

categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"범주형 컬럼: {categorical_cols}")
print(f"수치형 컬럼: {numerical_cols}")

# 3. 전처리 파이프라인 구성
# DropNone은 OneHotEncoder가 모든 범주를 유지
# handle_unknown='ignore'는 훈련 데이터에 없는 범주가 예측 단계에서 나타날 경우 무시
preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(handle_unknown='ignore', drop=None), categorical_cols)
], remainder='passthrough') # 수치형은 그대로 통과

# 4. 전체 파이프라인 구성
# 초기 RandomForestClassifier는 튜닝을 위한 기본 설정
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced')) # class_weight='balanced' 유지
])

# 5. 학습 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print(f"\n학습 데이터 크기 (X_train): {X_train.shape}")
print(f"테스트 데이터 크기 (X_test): {X_test.shape}")
print(f"학습 데이터의 슈퍼호스트 비율: {y_train.value_counts(normalize=True)}")

# --- 6. 하이퍼파라미터 그리드 정의 ---
# 'classifier__' 접두사를 사용하여 Pipeline 내의 classifier 단계의 파라미터를 지정
param_grid = {
    'classifier__n_estimators': [500, 1000, 1500], # 트리의 개수 (추가 탐색: 500, 1000, 1500)
    'classifier__max_depth': [20, 30, 40, None], # 트리의 최대 깊이 (None은 제한 없음)
    'classifier__min_samples_split': [2, 5, 10, 15], # 노드를 분할하기 위한 최소 샘플 수
    'classifier__min_samples_leaf': [1, 2, 5, 10],   # 리프 노드가 가져야 할 최소 샘플 수
    # 'classifier__class_weight': ['balanced', None] # 불균형이 심하다면 balanced 유지. 아니면 탐색.
                                                   # 여기서는 이미 'balanced'로 고정
}

# --- 7. GridSearchCV를 이용한 하이퍼파라미터 튜닝 ---
print("\n하이퍼파라미터 튜닝을 시작합니다. (시간이 오래 걸릴 수 있습니다...)")
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5, # 5-폴드 교차 검증
    scoring='f1', # 슈퍼호스트 예측이 중요하므로 f1-score를 최적화 지표로 선택
                  # 'accuracy', 'roc_auc' 등 다른 지표도 가능
    n_jobs=-1,    # 가능한 모든 CPU 코어 사용
    verbose=2     # 자세한 진행 상황 출력
)

grid_search.fit(X_train, y_train)

print("\n--- 하이퍼파라미터 튜닝 완료 ---")
print(f"최적의 하이퍼파라미터: {grid_search.best_params_}")
print(f"최적의 F1-Score: {grid_search.best_score_:.4f}")

# 최적의 모델 (파이프라인) 가져오기
best_pipeline = grid_search.best_estimator_

# --- 8. 최적 모델 평가 ---
print("\n--- 최적 모델 성능 평가 (테스트 세트) ---")
y_pred = best_pipeline.predict(X_test)
y_pred_proba = best_pipeline.predict_proba(X_test)[:, 1]

print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")

# --- 9. 최적 파이프라인 저장 ---
model_save_path = 'superhost_pipeline_rf_tuned.pkl' # 튜닝된 모델임을 명시
joblib.dump(best_pipeline, model_save_path)
print(f"\n최적화된 파이프라인 모델을 '{model_save_path}'로 저장했습니다.")



CSV 파일을 성공적으로 로드했습니다.
범주형 컬럼: ['host_about_length_group', 'room_type', 'name_length_group', 'description_length_group', 'host_has_profile_pic', 'host_identity_verified', 'is_long_term']
수치형 컬럼: ['amenities_cnt', 'availability_365', 'price', 'host_response_time_score', 'type_amenity_score', 'common_amenity_score', 'host_acceptance_rate_score', 'accommodates']

학습 데이터 크기 (X_train): (17846, 15)
테스트 데이터 크기 (X_test): (4462, 15)
학습 데이터의 슈퍼호스트 비율: host_is_superhost
0    0.72498
1    0.27502
Name: proportion, dtype: float64

하이퍼파라미터 튜닝을 시작합니다. (시간이 오래 걸릴 수 있습니다...)
Fitting 5 folds for each of 192 candidates, totalling 960 fits


C:\Users\HY\AppData\Local\Temp\ipykernel_18292\3076542489.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype(object) # object 타입으로 변환하여 categorical_cols에 포함되도록 함
C:\Users\HY\AppData\Local\Temp\ipykernel_18292\3076542489.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype(object) # object 타입으로 변환하여 categorical_cols에 포함되도록 함
C:\Users\HY\AppData\Local\Temp\ipykernel_18292\3076542489.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [1]:
import pandas as pd
import numpy as np
import joblib

# GridSearchCV 대신 RandomizedSearchCV 임포트
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# --- 0. 데이터 로드 ---
csv_path = 'superhost.csv'
try:
    df = pd.read_csv(
        csv_path,
        header=0,
        index_col='id',
        encoding='utf-8-sig'
    )
    print("CSV 파일을 성공적으로 로드했습니다.")
except FileNotFoundError:
    print(f"오류: '{csv_path}' 파일을 찾을 수 없습니다. 경로를 확인해주세요.")
    exit()

# 1. 타겟 및 피처 정의
TARGET = 'host_is_superhost'

strategy_cols = [
    'amenities_cnt', 'availability_365', 'price',
    'host_about_length_group', 'room_type', 'name_length_group', 'description_length_group',
    'host_has_profile_pic', 'host_response_time_score', 'type_amenity_score', 'common_amenity_score',
    'host_acceptance_rate_score', 'host_identity_verified', 'is_long_term', 'accommodates'
]

X = df[strategy_cols]
y = df[TARGET].astype(int)

# 2. 열 타입 분리 (OneHotEncoder가 bool 타입을 object로 인식하도록 명시적 변환)
for col in ['host_has_profile_pic', 'host_identity_verified', 'is_long_term']:
    if col in X.columns:
        X[col] = X[col].astype(object)

categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"범주형 컬럼: {categorical_cols}")
print(f"수치형 컬럼: {numerical_cols}")

# 3. 전처리 파이프라인 구성
preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(handle_unknown='ignore', drop=None), categorical_cols)
], remainder='passthrough')

# 4. 전체 파이프라인 구성
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced'))
])

# 5. 학습 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print(f"\n학습 데이터 크기 (X_train): {X_train.shape}")
print(f"테스트 데이터 크기 (X_test): {X_test.shape}")
print(f"학습 데이터의 슈퍼호스트 비율: {y_train.value_counts(normalize=True)}")

# --- 6. 하이퍼파라미터 그리드 정의 ---
# RandomizedSearchCV는 param_distributions 파라미터를 사용
param_dist = {
    'classifier__n_estimators': [500, 1000, 1500],
    'classifier__max_depth': [20, 30, 40, None],
    'classifier__min_samples_split': [2, 5, 10, 15],
    'classifier__min_samples_leaf': [1, 2, 5, 10],
}

# --- 7. RandomizedSearchCV를 이용한 하이퍼파라미터 튜닝 ---
print("\n하이퍼파라미터 튜닝을 시작합니다. (RandomizedSearchCV 사용 - 훨씬 빠름)")
random_search = RandomizedSearchCV( # <-- 여기를 RandomizedSearchCV로 변경
    pipeline,
    param_distributions=param_dist, # <-- 여기를 param_distributions로 변경
    n_iter=50,  
    cv=5,
    scoring='f1',
    n_jobs=2,  
    verbose=2
)

# 학습 시작
random_search.fit(X_train, y_train)

print("\n--- 하이퍼파라미터 튜닝 완료 ---")
print(f"최적의 하이퍼파라미터: {random_search.best_params_}")
print(f"최적의 F1-Score: {random_search.best_score_:.4f}")

# 최적의 모델 (파이프라인) 가져오기
best_pipeline = random_search.best_estimator_ # <-- random_search.best_estimator_ 사용

# --- 8. 최적 모델 평가 ---
print("\n--- 최적 모델 성능 평가 (테스트 세트) ---")
y_pred = best_pipeline.predict(X_test)
y_pred_proba = best_pipeline.predict_proba(X_test)[:, 1]

print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")

# --- 9. 최적 파이프라인 저장 ---
model_save_path = 'superhost_pipeline_rf_tuned.pkl'
joblib.dump(best_pipeline, model_save_path)
print(f"\n최적화된 파이프라인 모델을 '{model_save_path}'로 저장했습니다.")

CSV 파일을 성공적으로 로드했습니다.
범주형 컬럼: ['host_about_length_group', 'room_type', 'name_length_group', 'description_length_group', 'host_has_profile_pic', 'host_identity_verified', 'is_long_term']
수치형 컬럼: ['amenities_cnt', 'availability_365', 'price', 'host_response_time_score', 'type_amenity_score', 'common_amenity_score', 'host_acceptance_rate_score', 'accommodates']

학습 데이터 크기 (X_train): (17846, 15)
테스트 데이터 크기 (X_test): (4462, 15)
학습 데이터의 슈퍼호스트 비율: host_is_superhost
0    0.72498
1    0.27502
Name: proportion, dtype: float64

하이퍼파라미터 튜닝을 시작합니다. (RandomizedSearchCV 사용 - 훨씬 빠름)
Fitting 5 folds for each of 50 candidates, totalling 250 fits


C:\Users\HY\AppData\Local\Temp\ipykernel_21284\457813262.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype(object)
C:\Users\HY\AppData\Local\Temp\ipykernel_21284\457813262.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype(object)
C:\Users\HY\AppData\Local\Temp\ipykernel_21284\457813262.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i


--- 하이퍼파라미터 튜닝 완료 ---
최적의 하이퍼파라미터: {'classifier__n_estimators': 1500, 'classifier__min_samples_split': 10, 'classifier__min_samples_leaf': 2, 'classifier__max_depth': 20}
최적의 F1-Score: 0.6455

--- 최적 모델 성능 평가 (테스트 세트) ---
Accuracy: 0.8005
Precision: 0.6262
Recall: 0.6813
F1-Score: 0.6526
ROC-AUC: 0.8657

최적화된 파이프라인 모델을 'superhost_pipeline_rf_tuned.pkl'로 저장했습니다.
